In [1]:
from mltrainer import Trainer, TrainerSettings, ReportTypes, metrics
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor
import torch

import torch.optim as optim
optimizer = optim.Adam

import sys 
import os
sys.path.append(os.path.abspath('../networks'))
sys.path.append(os.path.abspath('../dev'))

In [5]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()

batchsize = 32

streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()


2025-06-04 16:59:09.955 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2025-06-04 16:59:09.956 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [ ]:
from pathlib import Path
from CNN import CNN

log_dir = Path("modellog").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)

accuracy = metrics.Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

model = CNN()

settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    earlystop_kwargs={
        "save": False,
        "verbose": True,
        "patience": 5,
    },
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )

2025-06-04 16:59:42.868 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to /home/azureuser/machinelearning-melissa/notebooks/modellog/20250604-165942
2025-06-04 16:59:42.870 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [7]:
trainer.loop()

100%|██████████| 1875/1875 [00:38<00:00, 48.78it/s]
2025-06-04 17:00:37.188 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.3803 test 0.3236 metric ['0.8796']
100%|██████████| 1875/1875 [00:39<00:00, 47.23it/s]
2025-06-04 17:01:18.824 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.2685 test 0.3048 metric ['0.8871']
100%|██████████| 1875/1875 [00:39<00:00, 47.79it/s]
2025-06-04 17:02:00.004 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.2266 test 0.2702 metric ['0.9029']
100%|██████████| 1875/1875 [00:39<00:00, 47.38it/s]
2025-06-04 17:02:41.652 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.2008 test 0.2699 metric ['0.9067']
100%|██████████| 1875/1875 [00:44<00:00, 41.91it/s]
2025-06-04 17:03:28.951 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.1760 test 0.2536 metric ['0.9121']
100%|██████████| 1875/1875 [00:44<00:00, 42.18it/s]
2025-06-04 17:04:15.434 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 0.1572 test